In [1]:
import sys
sys.path.append('..')

import os
import os.path as osp
import time
import pickle
import numpy as np
import pandas as pd
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TerminateOnNaN, TensorBoard
from sklearn.model_selection import train_test_split

# custom modules
from libs.utilities import load_dataset, create_autoencoder_model, load_autoencoder_model, load_autoencoder_lossfunc
from libs.model_utils import LossLayer
from libs.data_generator import DataGenerator
from libs.processing import pink_noise, s_to_power

# os.getcwd()

Using TensorFlow backend.
/home/christie/miniconda3/envs/denoising/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [17]:
#### PARAMETERS ####

model_source = '../models/conv12.jsont'


In [18]:
#### OTHER VARS ####
# loss function: data slice under consideration
time_slice = slice(None)
initial_epoch = 0
input_shape = (256, 16, 1)
template_args = {"dropout_rate": 0.2, "channels": 1, "activ_func": "relu"}

In [19]:
### SHOW MODEL FILE ###
import json
import pprint 

# pprint.pprint(json.load(open(model_source)))

In [20]:
### TRY AND GENERATE MODEL ###
# NOTE: restart the kernel and rerun everything after editing an imported script or lib
model, lossfunc = create_autoencoder_model(
    model_source, input_shape, template_args, time_slice=time_slice)

[u] Creating autoencoder model from ../models/conv12.jsont
[u] Model factory parameters: {'input_shape': (256, 16, 1), 'template_args': {'dropout_rate': 0.2, 'channels': 1, 'activ_func': 'relu'}, 'time_slice': slice(None, None, None)}
[m] Processing template...
[m] Adding layer CONV2D  -  {'filters': 32, 'kernel_size': 3, 'padding': 'same', 'strides': 2, 'bias_initializer': 'ones'}
[m] Adding layer BATCHNORM  -  {}
[m] Adding layer ACTIVATION  -  {'activation': 'relu'}
[m] Adding layer DROPOUT  -  {'rate': 0.2}
[m] Adding layer CONV2D  -  {'filters': 32, 'kernel_size': 3, 'padding': 'same', 'strides': 2, 'bias_initializer': 'ones'}
[m] Adding layer BATCHNORM  -  {}
[m] Adding layer ACTIVATION  -  {'activation': 'relu'}
[m] Adding layer DROPOUT  -  {'rate': 0.2}
[m] Adding layer CONV2D  -  {'filters': 64, 'kernel_size': 3, 'padding': 'same', 'strides': 2, 'bias_initializer': 'ones'}
[m] Adding layer BATCHNORM  -  {}
[m] Adding layer ACTIVATION  -  {'activation': 'relu'}
[m] Adding layer

In [21]:
### VERIFY GENERATED MODEL ###
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 256, 16, 1)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 128, 8, 32)        320       
_________________________________________________________________
batch_normalization_46 (Batc (None, 128, 8, 32)        128       
_________________________________________________________________
activation_46 (Activation)   (None, 128, 8, 32)        0         
_________________________________________________________________
dropout_43 (Dropout)         (None, 128, 8, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 64, 4, 32)         9248      
_________________________________________________________________
batch_normalization_47 (Batc (None, 64, 4, 32)         128       
__________

_____
# Can we use the built-in Keras Model exporters (dict, yaml, json)?

In [ ]:
### TEST KERAS BUILTIN JSON IMPORT/EXPORT
from models.model_example import AEModelFactory
obj = AEModelFactory(input_shape, 3, 16)
model2 = obj.get_model()

In [ ]:
model2.get_layer('encoder').summary()
model2.get_layer('decoder').summary()
model2.summary()


In [ ]:
from pprint import pprint
model_config = model2.get_config()
print(model_config)
#pprint(model_config)  ### looks prettier but takes all the space

In [ ]:
# well i guess that's a no